# Histogram2D time transfer

In [75]:
import time
import numpy as np
import asyncio as aio
import pandas as pd
from vega.widget import VegaWidget
import ipywidgets as widgets
nbins = 256
spec_no_data = {'$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 "width": 500,
 "height": 400,
 'data': {'name': 'data'},
 'encoding': {'color': {'field': 'z', 
                        'type': 'quantitative',
                        # "scale": {
                        # "domain": [0,1]  # Remove if domain changes
                        # }
                        },
  'x': {'field': 'x', 'type': 'ordinal'},
  'y': {'field': 'y', 'type': 'ordinal'}},
 'mark': 'rect',
 "config": {
    "axis": {
      "disable": True  # Change to False to see the ticks
    }
  }}
start = 0
desc = ""
output2 = widgets.Output()
def on_value_change(change):
    global start, desc
    with output2:
        received = change['new']
        print(desc, received - start)
widget = VegaWidget(spec=spec_no_data)
widget.observe(on_value_change, names='rec_time')
steps = 1
display(widget, output2)
nsamples=100_000
means = [0.1, 0.3]
cov = [[0.01, 0], [0, 0.09]]
rdata = np.random.multivariate_normal(means, cov, size=(nsamples))
h, *_ = np.histogram2d(rdata[:,0],rdata[:,1], bins=nbins, range=((-1.0,1.0),(-1.0,1.0)))
hist = h.astype('uint32')
vega_data = []
for i in range(hist.shape[0]):
    for j in range(hist.shape[1]):
        vega_data.append(dict(x=i, y=j, z=hist[i,j]))

widget._displayed = True

VegaWidget()

Output()

## Update with vega formated data

In [78]:
start = int(time.time()*1000)
desc = "Transfer time with vega formated data:"
widget.update('data', insert=vega_data, remove="true")

## Update with ipytablewidgets

In [81]:
start = int(time.time()*1000)
desc = "Transfer time with ipytablewidgets:"
widget.update('data', insert=hist, remove="true")

## Update with ipytablewidgets + LZ4 compression

In [84]:
start = int(time.time()*1000)
desc = "Transfer time with ipytablewidgets + LZ4:"
widget.compression = "lz4"
widget.update('data', insert=hist, remove="true")

## Update with ipytablewidgets + ZLIB compression

In [87]:
start = int(time.time()*1000)
desc = "Transfer time with ipytablewidgets + ZLIB:"
widget.compression = "zlib"
widget.update('data', insert=hist, remove="true")